In [1]:
from cval import CVal

cval = CVal(
    config_file="../config.toml",
    env_file="../.env"
)

Embeddings have been explicitly disabled. Using MockEmbedding.
LLM is explicitly disabled. Using MockLLM.


[06/22/24 22:13:24] INFO     HTTP Request: GET                                                      ]8;id=948341;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=992040;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             http://127.0.0.1:8080/v1/.well-known/openid-configuration "HTTP/1.1                   
                             404 Not Found"                                                                        

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"      ]8;id=326209;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=567504;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

                    INFO     HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 ]8;id=74960;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=139645;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             200 OK"                                                                               

In [2]:
cval.init_rag()

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/schema/TechDocs "HTTP/1.1   ]8;id=61618;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=83820;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             404 Not Found"                                                                        

                    INFO     HTTP Request: POST http://127.0.0.1:8080/v1/schema "HTTP/1.1 200 OK"   ]8;id=167253;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=528258;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/schema/TechDocs "HTTP/1.1   ]8;id=790757;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=260422;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             200 OK"                                                                               

                    INFO     HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/TechDocs          ]8;id=466488;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=720820;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     Successfully deleted index 'TechDocs'.                                     ]8;id=770361;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/llama_index/vector_stores/weaviate/base.py\base.py]8;;\:]8;id=152859;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/llama_index/vector_stores/weaviate/base.py#286\286]8;;\

...

[06/22/24 22:13:48] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/nodes "HTTP/1.1 200 OK"     ]8;id=696382;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=256627;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

[06/22/24 22:13:49] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/nodes "HTTP/1.1 200 OK"     ]8;id=393224;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=376580;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

[06/22/24 22:13:50] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/nodes "HTTP/1.1 200 OK"     ]8;id=531908;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=708634;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

[06/22/24 22:13:51] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/nodes "HTTP/1.1 200 OK"     ]8;id=292933;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=107392;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

[06/22/24 22:13:52] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/nodes "HTTP/1.1 200 OK"     ]8;id=207417;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=412724;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

In [3]:
from data import Dependency
from llama_index.core.base.response.schema import Response

dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)

response = cval.query(dependency=dep)

print(response, type(response))

[06/22/24 22:13:54] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=724028;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=858145;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/22/24 22:13:55] INFO     PyTorch version 2.3.1 available.                                          ]8;id=682443;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/datasets/config.py\config.py]8;;\:]8;id=224062;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/datasets/config.py#58\58]8;;\

/home/ssimon/github/cval/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[06/22/24 22:13:56] INFO     HTTP Request: GET http://127.0.0.1:8080/v1/schema/Tech "HTTP/1.1 200   ]8;id=458191;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=720452;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

                    INFO     HTTP Request: GET                                                      ]8;id=768170;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=128139;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             http://127.0.0.1:8080/v1/.well-known/openid-configuration "HTTP/1.1                   
                             404 Not Found"                                                                        

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"      ]8;id=780639;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=986752;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\

                    INFO     HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 ]8;id=627371;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=233666;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             200 OK"                                                                               

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/schema/Tech "HTTP/1.1 200   ]8;id=851904;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=220934;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

                    INFO     HTTP Request: GET http://127.0.0.1:8080/v1/schema/Tech "HTTP/1.1 200   ]8;id=595593;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=767408;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

Empty Response <class 'llama_index.core.base.response.schema.Response'>


In [4]:
response.response

'Empty Response'

In [5]:
response.source_nodes

[]